<font size="5">Ingest Time Series - GHG total with LULUCF, in kt CO₂ equivalent into Trino pipeline</font>


In [1]:
from dotenv import dotenv_values, load_dotenv
import osc_ingest_trino as osc
import os
import pathlib

Load Environment Variables

In [2]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [24]:
# use a catalog that is configured for iceberg
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'pcaf_sovereign_footprint'
ingest_table = 'sf_unfccc_with_lulucf'
ingest_country_table ='sf_unfccc_countries'

In [4]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

trino_bucket = osc.attach_s3_bucket("S3_DEV")

In [5]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

Open a Trino connection using JWT for authentication

In [6]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

('aicoe_osc_demo_results',)
('default',)
('demo_dv',)
('dera',)
('essd',)
('iceberg_demo',)
('information_schema',)
('ingest',)
('mdt_sandbox',)
('pcaf_sovereign_footprint',)
('sandbox',)
('wri_gppd',)


In [7]:
pip install country_converter --upgrade


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Load PCAF_UNFCC_WITH_LULUCF

In [8]:
import pandas as pd
import ParseXLS as parser

ticker_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'PCAF-sovereign-footprint/UNFCC/Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_equivalent.xlsx')
ticker_file.download_file(f'/tmp/Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_equivalent.xlsx')

df1 = parser.process('UNFCCC_with_LULUCF.ini','UNFCCC_with_LULUCF.csv') 




ticker_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'PCAF-sovereign-footprint/UNFCC/Annual_Net_emissions_removals_in_Gg_CO₂_equivalent_incl_LULUCF.csv')
ticker_file.download_file(f'/tmp/Annual_Net_emissions_removals_in_Gg_CO₂_equivalent_incl_LULUCF.csv')

df2 = parser.process('UNFCCC_with_LULUCF_other.ini','UNFCCC_with_LULUCF_other.csv') 

# combine both dataframes

df =pd.concat([df1,df2])

#pd.to_numeric(df["validity_date"],errors='raise')
df=df.astype({'validity_date': 'int32'})
df=df.astype({'country_iso_code': 'string'})
df=df.astype({'value': 'float'})

df= df.convert_dtypes()
df.info(verbose=True)
df= df[['rec_source','data_provider','country_iso_code','country_name','validity_date','attribute','value','value_units']].dropna(subset=['value'])
#df


European Union (Convention) not found in regex
European Union (KP) not found in regex
nan not found in ISO3
Source: UNFCCC GHG Data Interface not found in regex


UNFCCC_with_LULUCF.ini
file_list:
['/tmp/Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_equivalent.xlsx']
/tmp/Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_equivalent.xlsx
2
xls
/tmp/Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_equivalen
['0', '2']
eval_components
['Time Series - GHG total with LULUCF, in kt CO₂ equivalent']
['Party', 'Base year', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', 'Last Inventory Year (2019)', 'Change from base year to latest reported year']
Time Series - GHG total with LULUCF, in kt CO₂ equivalent
eval_components
['kt CO2e']
['Party', 'Base year', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', 'La

Note 1: The reporting and review requirements for GHG inventories are different for Annex I and non-Annex I Parties. The definition format of data for emissions/removals from the forestry sector is different for Annex I and non-Annex I Parties. not found in regex
Note 2: Base year data in the data interface relate to the base year under the Climate Change Convention (UNFCCC).  The base year under the Convention is defined slightly different than the base year under the Kyoto Protocol.  An exception is made for European Union (KP) whereby the base year under the Kyoto Protocol is displayed. not found in regex
Note 3: – means "No data available" not found in regex
Note 4: Data displayed on the data interface are "as received" from Parties. The publication of Party submissions on this website does not imply the expression of any opinion whatsoever on the part of the UNFCCC or the Secretariat of the United Nations concerning the legal status of any country, territory, city or area or of it

52
before melt
                                         country_name          1990  \
0                                           Australia  6.154780e+05   
1                                             Austria  6.622424e+04   
2                                             Belarus  1.084728e+05   
3                                             Belgium  1.423167e+05   
4                                            Bulgaria  8.084966e+04   
5                                              Canada  5.447072e+05   
6                                             Croatia  2.493960e+04   
7                                              Cyprus  5.352055e+03   
8                                             Czechia  1.901111e+05   
9                                             Denmark  7.760628e+04   
10                                            Estonia  3.808575e+04   
11                        European Union (Convention)  5.464793e+06   
12                                European Union (KP)  5.48044

Note 1: The reporting and review requirements for GHG inventories are different for Annex I and non-Annex I Parties. The definition format of data for emissions/removals from the forestry sector is different for Annex I and non-Annex I Parties. not found in regex
Note 2: Base year data in the data interface relate to the base year under the Climate Change Convention (UNFCCC).  The base year under the Convention is defined slightly different than the base year under the Kyoto Protocol.  An exception is made for European Union (KP) whereby the base year under the Kyoto Protocol is displayed. not found in regex
More then one regular expression match for Note 3: Some non-Annex I Parties submitted their GHG inventory data using  the format of the 2006 IPCC Guidelines in reporting GHG emissions/removals.  For this reason, these data could not be included in the data interface.  However, the data are available in the national communications (Andorra, Angola,  Antigua and Barbuda, Armenia, Aze

156
before melt
                                          country_name               1990  \
0                                         Party \ Unit  Gg CO₂ equivalent   
1                                          Afghanistan                NaN   
2                                              Albania            7833.16   
3                                              Algeria                NaN   
4                                               Angola                NaN   
..                                                 ...                ...   
151  Note 3: Some non-Annex I Parties submitted the...               None   
152                Note 4: – means "No data available"               None   
153  Note 5: Data displayed on the data interface a...               None   
154                  Source: UNFCCC GHG Data Interface               None   
155  Report produced on Friday, 10 June 2022, 14:51...               None   

                  1991               1992               199

In [15]:
df_annex1=df1[["country_iso_code"]].drop_duplicates()
df_annex1=df_annex1.astype({'country_iso_code': 'string'})
df_annex1["annex1_flag"] = "Y"
df_annex1=df_annex1.astype({'annex1_flag': 'string'})
df_annex1.info(verbose=True)
df_annex1

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43 entries, 0 to 44
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   country_iso_code  43 non-null     string
 1   annex1_flag       43 non-null     string
dtypes: string(2)
memory usage: 1.0 KB


,country_iso_code,annex1_flag
0,AUS,Y
1,AUT,Y
2,BLR,Y
3,BEL,Y
4,BGR,Y
5,CAN,Y
6,HRV,Y
7,CYP,Y
8,CZE,Y
9,DNK,Y


In [18]:
df_non_annex1=df2[["country_iso_code"]].drop_duplicates()
df_non_annex1["annex1_flag"] = "N"
df_non_annex1

,country_iso_code,annex1_flag
1,AFG,N
2,ALB,N
3,DZA,N
4,AGO,N
5,ATG,N
...,...,...
145,VNM,N
146,YEM,N
147,ZMB,N
148,ZWE,N


In [21]:
df_country = pd.concat([df_annex1,df_non_annex1])
df_country=df_country.astype({'country_iso_code': 'string'})
df_country = df_country.convert_dtypes()
df_country
#df_country[df_country["annex1_flag"]=='Y']

,country_iso_code,annex1_flag
0,AUS,Y
1,AUT,Y
2,BLR,Y
3,BEL,Y
4,BGR,Y
...,...,...
145,VNM,N
146,YEM,N
147,ZMB,N
148,ZWE,N


In [25]:
import osc_ingest_trino as osc
columnschema = osc.create_table_schema_pairs(df) 

sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_country_table}
"""
print(sql)
qres = engine.execute(sql)


drop table if exists osc_datacommons_dev.pcaf_sovereign_footprint.sf_unfccc_countries



In [26]:
print(ingest_catalog)
#df=df.drop(df[df.country_name=="cote d'ivoire"].index)
df_country.to_sql(ingest_country_table,
           con=engine,
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 1000, verbose = True))

osc_datacommons_dev
constructed fully qualified table name as: "pcaf_sovereign_footprint.sf_unfccc_countries"
inserting 192 records
  ('AUS', 'Y')
  ('AUT', 'Y')
  ('BLR', 'Y')
  ...
  ('[''AFG'', ''ALB'', ''AND'', ''AGO'', ''ATG'', ''ARG'', ''ARM'', ''AZE'', ''BHR'', ''BGD'', ''BLZ'', ''BEN'', ''BTN'', ''BRA'', ''BRN'', ''CPV'', ''KHM'', ''CHL'', ''COL'', ''COK'', ''CRI'', ''CIV'', ''CUB'', ''DOM'', ''EGY'', ''SLV'', ''GNQ'', ''SWZ'', ''FJI'', ''GAB'', ''GMB'', ''GHA'', ''GRD'', ''GNB'', ''HND'', ''IDN'', ''IRN'', ''JAM'', ''JOR'', ''KWT'', ''LAO'', ''LSO'', ''LBR'', ''MKD'', ''MYS'', ''MRT'', ''MUS'', ''MDA'', ''MNG'', ''MAR'', ''NAM'', ''NPL'', ''NIC'', ''NGA'', ''OMN'', ''PAN'', ''PNG'', ''PRY'', ''PER'', ''RWA'', ''SLE'', ''SGP'', ''SOM'', ''ZAF'', ''LCA'', ''SUR'', ''TJK'', ''THA'', ''TLS'', ''TGO'', ''TTO'', ''TUN'', ''UGA'', ''ARE'', ''URY'', ''UZB'', ''VUT'', ''VEN'', ''ZMB'']', 'N')
batch insert result: [(192,)]


In [27]:
import osc_ingest_trino as osc
columnschema = osc.create_table_schema_pairs(df) 

sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
print(sql)
qres = engine.execute(sql)


drop table if exists osc_datacommons_dev.pcaf_sovereign_footprint.sf_unfccc_with_lulucf



In [29]:
tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
    format = 'ORC',
    partitioning = array['country_iso_code']
)
"""
print(tabledef)
qres = engine.execute(tabledef)
#print(qres.fetchall())


create table if not exists osc_datacommons_dev.pcaf_sovereign_footprint.sf_unfccc_with_lulucf(
    rec_source varchar,
    data_provider varchar,
    country_iso_code varchar,
    country_name varchar,
    validity_date integer,
    attribute varchar,
    value double,
    value_units varchar
) with (
    format = 'ORC',
    partitioning = array['country_iso_code']
)



In [ ]:
# Delete all data from our db, so we start with empty table
sql=f"""
delete from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
qres = engine.execute(sql)
print(qres.fetchall())

In [ ]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
pd.read_sql(sql, engine)


In [30]:
print(ingest_catalog)
#df=df.drop(df[df.country_name=="cote d'ivoire"].index)
df.to_sql(ingest_table,
           con=engine,
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 1000, verbose = True))

osc_datacommons_dev
constructed fully qualified table name as: "pcaf_sovereign_footprint.sf_unfccc_with_lulucf"
inserting 1000 records
  ('Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_equivalent.xlsx', 'UNFCCC', 'AUS', 'Australia', 1990, 'Time Series - GHG total with LULUCF, in kt CO₂ equivalent', 615477.9858152439, 'kt CO2e')
  ('Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_equivalent.xlsx', 'UNFCCC', 'AUT', 'Austria', 1990, 'Time Series - GHG total with LULUCF, in kt CO₂ equivalent', 66224.24076499307, 'kt CO2e')
  ('Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_equivalent.xlsx', 'UNFCCC', 'BLR', 'Belarus', 1990, 'Time Series - GHG total with LULUCF, in kt CO₂ equivalent', 108472.81586531184, 'kt CO2e')
  ...
  ('Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_equivalent.xlsx', 'UNFCCC', 'EST', 'Estonia', 2013, 'Time Series - GHG total with LULUCF, in kt CO₂ equivalent', 18449.538821685383, 'kt CO2e')
batch insert result: [(1000,)]
inserting 1000 records
  ('Time_Series_GHG_total_with_LULUCF

In [ ]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table} where country_iso_code='CAN' and validity_date=2019"""
pd.read_sql(sql, engine)
